In [1]:
import os
os.system(r'python C:\users\barby\documents\github\runs\upload_activities.py')
import psycopg2
import datetime

import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure
from bokeh.palettes import all_palettes
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import DatetimeTickFormatter, Range1d, WheelZoomTool, HoverTool, ColumnDataSource, CustomJS
from bokeh.models import Arrow, VeeHead, Label, Text, Legend, BoxAnnotation

%matplotlib inline

km_in_a_mile = 1.60934

def readConfig(key):
    config = pd.read_table(r'c:\users\barby\documents\config.txt', header = None)
    config = [c.split('=') for c in config[0]]
    out = [c[1] for c in config if c[0] == key][0]
    return(out)

def dbGetQuery(q):
    pw = readConfig('pw')
    conn_string = "host='kavdb.c9lrodma91yx.us-west-2.rds.amazonaws.com' dbname='kavdb' user='lkavenagh' password='" + pw + "'"
    conn = psycopg2.connect(conn_string)
    conn.autocommit = True
    dat = pd.read_sql(q, conn)
    conn.close()
    return(dat)

In [2]:
reset_output()
output_notebook()

Loading BokehJS ...

# YTD activities

In [3]:
dat = dbGetQuery("SELECT * FROM runs.activities")
dat = dat.sort_values('start_date')

dat = dat.append(dat.tail(1)).reset_index(drop = True)
dat.loc[len(dat)-1, 'start_date'] = datetime.datetime.now()
dat.loc[len(dat)-1, 'distance'] = 0

dat['year'] = [c.year for c in dat.start_date]
dat['doy'] = [c.timetuple().tm_yday for c in dat.start_date]

dat['distance_miles'] = [(c/1000)/km_in_a_mile for c in dat.distance]
dat['ytd_distance'] = dat.groupby('year', as_index = False).cumsum()['distance_miles']
dat['generic_date'] = [datetime.date(1900,c.month,c.day) for c in dat.start_date]


In [4]:
target = 800
actual = dat.loc[dat.start_date == max(dat.start_date), 'ytd_distance'].item()

plot_dat = dat[['generic_date', 'year', 'ytd_distance']]
plot_dat = plot_dat.groupby(['generic_date', 'year'], as_index = False).max()
plot_dat = plot_dat.pivot_table(index = 'generic_date', columns = 'year')
plot_dat.columns = plot_dat.columns.droplevel().rename(None)
plot_dat = plot_dat.reset_index()
plot_dat = plot_dat.apply(lambda series: series.loc[:series.last_valid_index()].ffill())

x = range(len(plot_dat))
y = [(target / len(plot_dat)) * x for x in x]
plot_dat['target'] = y
t = datetime.datetime.today().date()
today_target = plot_dat.loc[plot_dat.generic_date == datetime.date(1900, t.month, t.day), 'target'].item()

p = figure(plot_width=800, 
            plot_height=400, 
            title = str(datetime.datetime.now().date()) + ' - YTD miles |'
            ' Target: ' + str(round(today_target,1)) + ' miles |' + 
            ' Actual: ' + str(round(actual,1)) + ' miles |' +
            ' Surplus: ' + str(round(actual - today_target,1)) + ' miles', 
            tools="pan,wheel_zoom,box_zoom,reset", 
            active_scroll = 'wheel_zoom',
            x_axis_type = 'datetime')

p.x_range = Range1d(datetime.date(1899,12,31),datetime.date(1900,12,31))

plot_dat = plot_dat.reset_index(drop = True)
plot_dat.columns = [str(c) for c in plot_dat.columns]

mypalette = all_palettes['Category10'][len(plot_dat.columns)-1]

plot_dat['datestr'] = [c.strftime('%d %b') for c in plot_dat['generic_date']]

source = ColumnDataSource(data = plot_dat)

legend_it = []
to_plot = plot_dat.columns[1:-1]
for i,l in enumerate(to_plot):
    if l == 'target':
        c = p.line(x = 'generic_date',
            y = l,
            line_color = 'black',
            line_dash = 'dashed',
            line_width = 2,
            source = source)
    elif i == (len(to_plot)-2):
        c = p.line(x = 'generic_date',
            y = l,
            line_color = mypalette[i],
            line_width = 4,
            source = source)
    elif i > (len(to_plot) - 5):
        c = p.line(x = 'generic_date',
            y = l,
            line_color = mypalette[i],
            line_width = 2,
            source = source)
    else:
        c = p.line(x = 'generic_date',
            y = l,
            line_color = mypalette[i],
            line_width = 2,
            visible = False,
            source = source)
    
    legend_it.append((l.title(), [c]))

p.xaxis.formatter = DatetimeTickFormatter(
        hours=["%B %d"],
        days=["%B %d"],
        months=["%B"],
        years=["%B"]
    )
p.xaxis.major_label_orientation = 3/4

legend = Legend(items = legend_it, location=(10, 120))
legend.click_policy = "hide"
p.add_layout(legend, 'right')

show(p)

html = file_html(p, CDN, "YTD totals")
text_file = open(r"C:\Users\barby\Documents\GitHub\runs\ytd_totals.html", "w")
text_file.write(html)
text_file.close()

220.28985507246378


# Monthly average pace

In [5]:
dat = dbGetQuery("SELECT start_date, average_speed, distance FROM runs.activities WHERE type = 'Run' and average_speed > 2.1")
dat = dat.sort_values('start_date')
dat['month'] = [datetime.date(c.year, c.month, 1) for c in dat.start_date]
dat['average_pace'] = [(km_in_a_mile/60)/((0.00001+c) / 1000) for c in dat.average_speed]
dat.distance = [(c/1000)/km_in_a_mile for c in dat.distance]

In [6]:
out = pd.DataFrame(columns = ['date', 'average_pace_7', 'average_pace_30', 'average_pace_YTD'])

for i in range(len(dat)):
    dat_7 = dat.loc[(dat.start_date < dat.start_date[i]) & (dat.start_date >= (dat.start_date[i] - datetime.timedelta(7)))]
    dat_30 = dat.loc[(dat.start_date < dat.start_date[i]) & (dat.start_date >= (dat.start_date[i] - datetime.timedelta(30)))]
    dat_YTD = dat.loc[(dat.start_date < dat.start_date[i]) & (dat.start_date >= datetime.date(dat.start_date[i].year,1,1))]

    pace_y = pace_30 = pace_YTD = None
    if len(dat_7) > 0:
        pace_7 = np.nansum(dat_7.distance * dat_7.average_pace) / np.nansum(dat_7.distance)
    if len(dat_30) > 0:
        pace_30 = np.nansum(dat_30.distance * dat_30.average_pace) / np.nansum(dat_30.distance)
    if len(dat_YTD) > 0:
        pace_YTD = np.nansum(dat_YTD.distance * dat_YTD.average_pace) / np.nansum(dat_YTD.distance)

    out = out.append(pd.DataFrame([[dat.start_date[i].date(),
                            pace_7,
                            pace_30,
                            pace_YTD]], columns = ['date', 'average_pace_7', 'average_pace_30', 'average_pace_YTD']
                         ))
out = out.sort_values('date')

In [8]:
p1 = figure(plot_width=800, 
           plot_height=400, 
           title = 'Monthly pace',
           tools="pan,wheel_zoom,box_zoom,reset", 
           active_scroll = 'wheel_zoom',
           x_axis_type = 'datetime')
p1.xaxis.formatter = DatetimeTickFormatter(
        hours=["%b-%Y"],
        days=["%b-%Y"],
        months=["%b-%Y"],
        years=["%b-%Y"]
    )

p1.y_range = Range1d(6,10)

legend_it = []
line = p1.line(x = out.date,
        y = out.average_pace_7,
        line_color = 'blue',
        line_width = 2,
        visible = False)
legend_it.append(('7 day average pace', [line]))

line = p1.line(x = out.date,
        y = out.average_pace_30,
        line_color = 'orange',
        line_width = 2,
        visible = True)
legend_it.append(('30 day average pace', [line]))

line = p1.line(x = out.date,
        y = out.average_pace_YTD,
        line_color = 'green',
        line_width = 2,
        visible = True)
legend_it.append(('YTD average pace', [line]))

legend = Legend(items = legend_it, location=(10, 120))
legend.click_policy = "hide"
p1.add_layout(legend, 'right')

p1.xaxis.major_label_orientation = 3/4

show(p1)

# Monthly elevation gain

In [9]:
dat = dbGetQuery("SELECT start_date, total_elevation_gain FROM runs.activities")
dat = dat.sort_values('start_date')
dat['month'] = [datetime.date(c.year, c.month, 1) for c in dat.start_date]
dat = dat.groupby('month', as_index = False).sum()
dat = dat.sort_values('month')
dat.month = [c.strftime('%b-%y') for c in dat.month]
dat = dat.reset_index(drop = True)

In [10]:
p1 = figure(plot_width=900, 
           plot_height=400, 
           title = 'Monthly elevation gain',
           tools="xpan,xwheel_zoom,box_zoom,reset", 
           active_scroll = 'xwheel_zoom',
           x_range=list(dat.month))

p1.xaxis.major_label_orientation = 3/4

line = p1.vbar(x = dat.month,
        top = dat.total_elevation_gain,
        line_color = 'blue',
        width = 0.75,
        visible = True)

p1.xgrid.grid_line_color = None
p1.y_range.start = 0

l = dat.loc[dat.month == 'May-14'].index.item()
r = dat.loc[dat.month == 'Jun-17'].index.item()
mid_box = BoxAnnotation(left = l, 
                        right = r, 
                        fill_alpha=0.1, fill_color='green')
p1.add_layout(mid_box)

mytext = Label(x = l + (r-l)/2, 
               y = 2000,
               text = 'San Francisco',
              text_align = 'center',
              text_font_style = 'bold')

p1.add_layout(mytext)

show(p1)

# Monthly mileage

In [11]:
dat = dbGetQuery("SELECT start_date, distance FROM runs.activities")
dat.distance = [(c/1000)/km_in_a_mile for c in dat.distance]
dat = dat.sort_values('start_date')
dat['month'] = [datetime.date(c.year, c.month, 1) for c in dat.start_date]
dat = dat.groupby('month', as_index = False).sum()
dat = dat.sort_values('month')
dat.month = [c.strftime('%b-%y') for c in dat.month]
dat = dat.reset_index(drop = True)

In [12]:
p1 = figure(plot_width=900, 
           plot_height=400, 
           title = 'Monthly distance (miles)',
           tools="xpan,xwheel_zoom,box_zoom,reset", 
           active_scroll = 'xwheel_zoom',
           x_range=list(dat.month))

p1.xaxis.major_label_orientation = 3/4

line = p1.vbar(x = dat.month,
        top = dat.distance,
        line_color = 'blue',
        width = 0.75,
        visible = True)

p1.xgrid.grid_line_color = None
p1.y_range.start = 0

l = dat.loc[dat.month == 'May-14'].index.item()
r = dat.loc[dat.month == 'Jun-17'].index.item()
mid_box = BoxAnnotation(left = l, 
                        right = r, 
                        fill_alpha=0.1, fill_color='green')
p1.add_layout(mid_box)

mytext = Label(x = l + (r-l)/2, 
               y = 150,
               text = 'San Francisco',
              text_align = 'center',
              text_font_style = 'bold')

p1.add_layout(mytext)

show(p1)